## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

os.environ['groq_api_key']=os.getenv("GROQ_API_KEY")




In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f5d7cee6450>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f5d7cb1e790>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 27, 'total_tokens': 82, 'completion_time': 0.11, 'prompt_time': 0.003535862, 'queue_time': None, 'total_time': 0.113535862}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-218d4449-8b1a-431c-926f-1eb903fecebf-0', usage_metadata={'input_tokens': 27, 'output_tokens': 55, 'total_tokens': 82})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and you are a Chief AI Engineer! \n\nIs there anything else you'd like me to remember about you?  \n\n", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 102, 'total_tokens': 137, 'completion_time': 0.07, 'prompt_time': 0.005276236, 'queue_time': None, 'total_time': 0.07527623600000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_e238d2cf52', 'finish_reason': 'stop', 'logprobs': None}, id='run-69203ab2-a54d-43ee-9676-f68367cdac82-0', usage_metadata={'input_tokens': 102, 'output_tokens': 35, 'total_tokens': 137})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
store={}
def get_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
from langchain_core.messages import HumanMessage, AIMessage
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [13]:
response.content

'Hello Krish! 👋 It’s great to meet you. How can I assist you today?'

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" We have context: user introduced themselves as "Krish". So answer: Krish. Should be concise.'}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 119, 'total_tokens': 165, 'completion_time': 0.094324358, 'prompt_time': 0.004444779, 'queue_time': 0.052996661, 'total_time': 0.098769137}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e88ce9c728', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--72b6b2c6-23ff-41b9-a8b2-fe7b48c500a1-0', usage_metadata={'input_tokens': 119, 'output_tokens': 46, 'total_tokens': 165})

In [17]:
## change the config-->session id
config1={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
   
)
response.content

'Your name is Krish.'

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Got it—nice to meet you, John! How can I help you today?'

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [33]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        ("human","{text}"),

        
    ]
)

chain=prompt|model|parser

In [34]:
chain.invoke({"text":"My name is Alice and I am a software developer"})

'Nice to meet you, Alice! As a software developer, you probably juggle a lot of interesting challenges. How can I assist you today?'

In [35]:
with_message_history=RunnableWithMessageHistory(chain,get_history)

In [ ]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    ({"text":"Hello, I am Bob and I work as a data scientist."}),
    config=config
)

response

'Hello again, Bob! 👋 Great to see you back. How can I assist you today?'

In [38]:
response = with_message_history.invoke(
    ({"text":"Can you remind me my name and profession?"}),
    config=config,
)

response

'Sure! Your name is **Bob**, and you work as a **data scientist**.'

In [41]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        ("human","{text}"),
    ]
)

chain = prompt | model|parser

In [42]:
response=chain.invoke({"text":"Hello, My name is Hari","language":"Hindi"})
response

'नमस्ते हरि! आपसे मिलकर खुशी हुई। मैं आपकी कैसे मदद कर सकता हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष! 😊 \n\nतुम्हें कैसे हो? क्या मैं तुमकी कोई मदद कर सकता हूँ?\n'

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'आपका नाम कृष है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [43]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    | parser
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

KeyError: "Input to ChatPromptTemplate is missing variables {'text'}.  Expected: ['language', 'text'] Received: ['messages', 'language']\nNote: if you intended {text} to be part of the string and not a variable, please escape it with double curly braces like: '{{text}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [ ]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"